In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from google.colab import drive
import torch.nn.init as init
import os

In [ ]:
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [ ]:
# with open('/drive/My Drive/BIA_MnistTrained/sample_images.pth', 'r') as f:
sample_images = torch.load('/drive/My Drive/BIA_MnistTrained/sample_images.pth')
sample_images = sample_images.unsqueeze(dim=1).type(torch.double)
print(sample_images.shape, sample_images.dtype)

torch.Size([10, 1, 28, 28]) torch.float64


In [ ]:
def get_activation(name):
    def hook(model, input, output):
        # print(input)
        # print(output)
        activation[name] = output.detach()
    return hook

In [ ]:
net = Net()
for name, module in net.named_children():
    print(name, module)

conv1 Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
conv2 Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
conv2_drop Dropout2d(p=0.5, inplace=False)
fc1 Linear(in_features=320, out_features=50, bias=True)
fc2 Linear(in_features=50, out_features=10, bias=True)


In [ ]:
model_num = 20
for i in range(19, model_num):
    at_dir = '/drive/My Drive/BIA_MnistTrained/'
    path = os.path.join(at_dir, 'model_{0}.pth').format(i)
    model_path = os.path.join(at_dir, 'models_layers/model_{0}').format(i)
    if not os.path.exists(model_path):
        os.mkdir(model_path)
    weights = torch.load(path)
    net = Net()
    net.load_state_dict(weights)
    image_num = sample_images.shape[0]
    activation = {}
    layers = {}
    for name, module in net.named_children():
        layers[name] = torch.tensor([], dtype=float)
    with torch.no_grad():
        for i in range(image_num):
            x = torch.tensor(sample_images[i, :, :, :].clone(), dtype=torch.float)
            for name, module in net.named_children():
                module.register_forward_hook(get_activation(name))
            net_out = net(x)
            for name, module in net.named_children():
                if layers[name].nelement() == 0:
                    layers[name] = torch.cat([layers[name], activation[name]], dim=0)
                elif layers[name].shape != activation[name].shape:
                    temp = activation[name].unsqueeze(0)
                    layers[name] = torch.cat([layers[name], temp], dim=0)
                else:
                    layers[name] = torch.stack([layers[name], activation[name]], dim=0)
    for name, module in net.named_children():
        print(os.path.join(model_path, name+'.pth'))
        torch.save(layers[name], os.path.join(model_path, name+'.pth'))
    break



/drive/My Drive/BIA_MnistTrained/models_layers/model_19/conv1.pth
/drive/My Drive/BIA_MnistTrained/models_layers/model_19/conv2.pth
/drive/My Drive/BIA_MnistTrained/models_layers/model_19/conv2_drop.pth
/drive/My Drive/BIA_MnistTrained/models_layers/model_19/fc1.pth
/drive/My Drive/BIA_MnistTrained/models_layers/model_19/fc2.pth


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
